# imports

In [1]:
from mylib import dataset

from dotenv import dotenv_values
from pathlib import Path
from pprint import pprint, pformat

import pandas as pd
import pandas.api.types as pd_types

import plotly.express as plotly_px

import joblib

In [2]:
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 50) # Устанавливаем максимальное количество отображаемых столбцов равным 50
#pd.set_option('display.max_rows', 50) # Устанавливаем максимальное количество отображаемых строк равным 20
pd.options.display.float_format = '{:.5f}'.format # Устанавливаем формат отображения чисел с двумя знаками после запятой
pd.options.mode.use_inf_as_na = True # Настройка режима Pandas для рассмотрения бесконечностей (inf) как пропущенных значений (NA)

# Конфигурация формата отображения графиков в виде векторных изображений
%config InlineBackend.figure_format = 'svg'

# для построения графиков внутри Jupyter Notebook
%matplotlib inline

# 1. Загрузка конфигов

In [3]:
# загрузить параметры
settings = {
    **dotenv_values("settings")
}
dataset_folder = str(Path(Path.cwd(), settings["DATASET_SUBFOLDER"]))
result_folder = str(Path(Path.cwd(), settings["RESULT_SUBFOLDER"]))

In [4]:
settings

{'DATASET_SUBFOLDER': 'dataset',
 'RESULT_SUBFOLDER': 'result',
 'CACHE_SUBFOLDER': 'cached_results',
 'DATASET_FILENAME_AFTER_EDA': '01_dataset_df_after_EDA.joblib',
 'PARAMS_FILENAME_AFTER_EDA': '01_params_after_EDA.joblib',
 'DATASET_FILENAME_READY_FOR_TRAIN_ORIGINAL': '02_dataset_catencoder_df_ready_for_train_original.joblib',
 'PARAMS_FILENAME_READY_FOR_TRAIN_ORIGINAL': '02_params_ready_for_train_original.joblib',
 'GRID_SEARCH_TEMPLATE_FILENAME': '03_GridSearch_%s.pickle',
 'MODEL_CLASS_TEMPLATE_FILENAME': '04_model_%s.pickle'}

# 2. EDA датасета

In [5]:
dataset_df = pd.read_csv(Path(dataset_folder, 'train_dataset.csv')) #, index_col="id")

In [6]:
display(dataset_df.sample(7))
display(dataset_df.info())

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
8759,25,175,60,77.00000,0.90000,0.90000,1,1,125,75,92,164,303,76,27,15.10000,1,1.10000,38,28,14,0,0
36714,40,175,80,88.00000,1.20000,1.20000,1,1,120,85,98,283,202,63,179,16.30000,1,1.00000,31,56,74,0,0
12484,60,145,60,85.00000,1.20000,1.00000,1,1,120,78,97,240,138,78,134,14.20000,1,1.20000,116,141,86,0,0
28871,25,180,85,82.00000,0.90000,0.70000,1,1,160,90,94,184,151,46,107,15.30000,1,0.80000,42,70,140,0,1
18483,70,165,60,88.00000,0.60000,0.80000,1,1,147,78,117,139,88,59,62,14.40000,1,1.00000,23,19,49,0,1
5527,25,175,80,79.00000,0.50000,0.60000,1,1,139,79,78,134,44,48,77,15.60000,1,0.90000,34,36,43,1,1
31687,40,160,55,74.00000,1.00000,0.90000,1,1,118,70,92,242,63,52,177,13.20000,1,0.80000,11,11,19,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38984 entries, 0 to 38983
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  38984 non-null  int64  
 1   height(cm)           38984 non-null  int64  
 2   weight(kg)           38984 non-null  int64  
 3   waist(cm)            38984 non-null  float64
 4   eyesight(left)       38984 non-null  float64
 5   eyesight(right)      38984 non-null  float64
 6   hearing(left)        38984 non-null  int64  
 7   hearing(right)       38984 non-null  int64  
 8   systolic             38984 non-null  int64  
 9   relaxation           38984 non-null  int64  
 10  fasting blood sugar  38984 non-null  int64  
 11  Cholesterol          38984 non-null  int64  
 12  triglyceride         38984 non-null  int64  
 13  HDL                  38984 non-null  int64  
 14  LDL                  38984 non-null  int64  
 15  hemoglobin           38984 non-null 

None

In [7]:
# выделим числовые и категориальные переменные
params = {}
params["target_column"] = "smoking"
# columns_X - переменные датасета
params["columns_X"] = dataset_df.columns.to_list()
params["columns_X"].remove(params["target_column"])
params["num_columns"] = []
params["cat_columns"] = []
for col in params["columns_X"]:
    if dataset_df[col].nunique() <= 2 or not pd_types.is_numeric_dtype(dataset_df[col]):
        params["cat_columns"].append(col)
    else:
        params["num_columns"].append(col)
print(f'target_columns={params["target_column"]}')        
print(f'columns_X={params["columns_X"]}')
print(f'cat_columns={pformat(params["cat_columns"])}')
print(f'num_columns={pformat(params["num_columns"])}')

target_columns=smoking
columns_X=['age', 'height(cm)', 'weight(kg)', 'waist(cm)', 'eyesight(left)', 'eyesight(right)', 'hearing(left)', 'hearing(right)', 'systolic', 'relaxation', 'fasting blood sugar', 'Cholesterol', 'triglyceride', 'HDL', 'LDL', 'hemoglobin', 'Urine protein', 'serum creatinine', 'AST', 'ALT', 'Gtp', 'dental caries']
cat_columns=['hearing(left)', 'hearing(right)', 'dental caries']
num_columns=['age',
 'height(cm)',
 'weight(kg)',
 'waist(cm)',
 'eyesight(left)',
 'eyesight(right)',
 'systolic',
 'relaxation',
 'fasting blood sugar',
 'Cholesterol',
 'triglyceride',
 'HDL',
 'LDL',
 'hemoglobin',
 'Urine protein',
 'serum creatinine',
 'AST',
 'ALT',
 'Gtp']


In [8]:
print("Доля пропусков в столбцах в %:")
nan_in_columns = dataset.nans_percents(dataset_df)
# список столбцов с пропусками
params["columns_with_nan"] = nan_in_columns[nan_in_columns > 0].index.to_list()
nan_in_columns[nan_in_columns > 0]

Доля пропусков в столбцах в %:


Series([], dtype: float64)

In [9]:
dataset_df_describe_num = dataset.eda_df(dataset_df[params["num_columns"]+[params["target_column"]]])
display(dataset_df_describe_num)

,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),systolic,relaxation,fasting blood sugar,Cholesterol,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,smoking
count,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000,38984.00000
mean,44.12759,164.68949,65.93872,82.06212,1.01495,1.00877,121.47563,75.99441,99.34227,196.88349,126.74946,57.29315,115.08149,14.62426,1.08652,0.88603,26.19824,27.14519,39.90504,0.36728
std,12.06356,9.18751,12.89658,9.32680,0.49853,0.49381,13.64352,9.65873,20.64274,36.35394,71.80314,14.61782,42.88316,1.56653,0.40211,0.22062,19.17560,31.30994,49.69384,0.48207
min,20.00000,130.00000,30.00000,51.00000,0.10000,0.10000,71.00000,40.00000,46.00000,55.00000,8.00000,4.00000,1.00000,4.90000,1.00000,0.10000,6.00000,1.00000,2.00000,0.00000
10%,30.00000,150.00000,50.00000,70.00000,0.60000,0.60000,104.00000,64.00000,83.00000,152.00000,55.00000,41.00000,73.00000,12.70000,1.00000,0.60000,16.00000,12.00000,13.00000,0.00000
25%,40.00000,160.00000,55.00000,76.00000,0.80000,0.80000,112.00000,70.00000,89.00000,172.00000,74.00000,47.00000,91.00000,13.60000,1.00000,0.80000,19.00000,15.00000,17.00000,0.00000
50%,40.00000,165.00000,65.00000,82.00000,1.00000,1.00000,120.00000,76.00000,96.00000,195.00000,108.00000,55.00000,113.00000,14.80000,1.00000,0.90000,23.00000,21.00000,26.00000,0.00000
75%,55.00000,170.00000,75.00000,88.00000,1.20000,1.20000,130.00000,82.00000,104.00000,219.00000,160.00000,66.00000,136.00000,15.80000,1.00000,1.00000,29.00000,31.00000,44.00000,1.00000
90%,60.00000,175.00000,85.00000,94.00000,1.50000,1.50000,138.00000,88.00000,115.00000,244.00000,227.00000,77.00000,158.00000,16.50000,1.00000,1.10000,37.00000,47.00000,78.00000,1.00000
max,85.00000,190.00000,135.00000,129.00000,9.90000,9.90000,233.00000,146.00000,423.00000,445.00000,999.00000,359.00000,1860.00000,21.10000,6.00000,11.60000,1090.00000,2914.00000,999.00000,1.00000


In [10]:
print("Уникальные значения по столбцам")
nunique = dataset_df[params["columns_X"]].nunique()
for column in nunique.index:
    if nunique[column] <= 10:
        print(f'{column:20}: {nunique[column]:6}, {dataset_df[column].unique().tolist()}')
    else:
        print(f'{column:20}: {nunique[column]:6}')


Уникальные значения по столбцам
age                 :     14
height(cm)          :     13
weight(kg)          :     22
waist(cm)           :    545
eyesight(left)      :     19
eyesight(right)     :     17
hearing(left)       :      2, [1, 2]
hearing(right)      :      2, [1, 2]
systolic            :    125
relaxation          :     94
fasting blood sugar :    258
Cholesterol         :    279
triglyceride        :    389
HDL                 :    122
LDL                 :    286
hemoglobin          :    143
Urine protein       :      6, [1, 3, 2, 5, 4, 6]
serum creatinine    :     34
AST                 :    195
ALT                 :    230
Gtp                 :    439
dental caries       :      2, [1, 0]


In [11]:
dataset_df_describe_cat = dataset_df[params["cat_columns"]].describe()
display(dataset_df_describe_cat)

,hearing(left),hearing(right),dental caries
count,38984.00000,38984.00000,38984.00000
mean,1.02537,1.02619,0.21442
std,0.15725,0.15970,0.41043
min,1.00000,1.00000,0.00000
25%,1.00000,1.00000,0.00000
50%,1.00000,1.00000,0.00000
75%,1.00000,1.00000,0.00000
max,2.00000,2.00000,1.00000


Пропусков в данных нет.

In [12]:
# сохранить промежуточные результаты

_ = joblib.dump(dataset_df, Path(result_folder, settings["DATASET_FILENAME_AFTER_EDA"]), compress=3)
_ = joblib.dump(params, Path(result_folder, settings["PARAMS_FILENAME_AFTER_EDA"]), compress=3)

# 3. Доли значений целевой переменной

In [13]:
#print(dataset_df["smoking"].value_counts(normalize=True))
plotly_px.bar(dataset_df["smoking"].value_counts(normalize=True), 
              title="Доля вхождений классов в %",
              width=400, height=300)

# 4. Графики для анализа

In [14]:
fig = plotly_px.imshow(dataset_df.corr(), height=1000, width=1000, text_auto='.2f' )
fig.show()

Каких-то значимых корреляций в данных нет.

In [15]:
dataset.show_boxes_plt(dataset_df, params["num_columns"])

Почти во всех столбцах есть выбросы, если считать по +-1.5IQR. Но значения вполне могут быть адекватными данными.
Тем не менее подготовим два датасета, в которых выбросы заменены на:
  а) ближайшие крайние значения, входящие в +-1.5IQR
  б) на медиану

In [16]:
_ = joblib.dump(dataset_df, Path(result_folder, settings["DATASET_FILENAME_READY_FOR_TRAIN_ORIGINAL"]), compress=3)
_ = joblib.dump(params, Path(result_folder, settings["PARAMS_FILENAME_READY_FOR_TRAIN_ORIGINAL"]), compress=3)